In [ ]:
import os
import sys
# 项目列表

import json
import pandas as pd

from typing import List

from alibabacloud_devops20210625.client import Client as devops20210625Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient
from alibabacloud_devops20210625 import models as devops_20210625_models
# print(os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'], os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET'])
config = open_api_models.Config(
            access_key_id=os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'],
            access_key_secret=os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET']
        )
# Endpoint 请参考 https://api.aliyun.com/product/devops
config.endpoint = f'devops.cn-hangzhou.aliyuncs.com'
client = devops20210625Client(config)

In [ ]:
runtime = util_models.RuntimeOptions()
headers = {}
organization_id="6189f099041d450d2c253abc"
project_id='0c593b861aafc8b8546d67dd65'

work_item_condition='{"conditionGroups":[[{"fieldIdentifier":"status","operator":"CONTAINS","value":["28","30","100010","29","100085"],"toValue":null,"className":"status","format":"list"},{"fieldIdentifier":"assignedTo","operator":"CONTAINS","value":["64100f263cecdaec3ac85633","641928092ca4e1cd30e29329","6329342591e82b7db4870656","64100c307dcfe0feff493ad8","618a2310e6290b981efebcf4","63e0a756378edb0fd9c486e6","64daf5533b187bc174b88ccb","63b66d0aabc54e81b876dbe8"],"toValue":null,"className":"user","format":"list"},{"fieldIdentifier":"gmtCreate","operator":"BETWEEN","value":["2023-10-01 00:00:00"],"toValue":"2023-12-31 23:59:59","className":"dateTime","format":"input"},{"fieldIdentifier":"a9a03b39ac7309f506445bc54e","operator":"CONTAINS","value":["saas"],"toValue":null,"className":"string","format":"list"}]]}'
# '{"conditionGroups":[[{"fieldIdentifier":"status","operator":"CONTAINS","value":["63798bd5f6855ea51abcd1b0d6"],"toValue":null,"className":"status","format":"list"},{"fieldIdentifier":"workitemType","operator":"CONTAINS","value":["1a787d02dbffcf9bbe93e73631"],"toValue":null,"className":"workitemType","format":"list"}]]}'
req=devops_20210625_models.ListWorkitemsRequest(space_type='Project',
                                                category='Bug',
                                                conditions=work_item_condition,
                                                max_results=100,
                                                search_type="LIST",
                                              space_identifier=project_id)
res=client.list_workitems(organization_id=organization_id,
                                   request=req)

print(res.body.to_map())
items_array = res.body.to_map()['workitems']

# 将这段代码转成小方法，参数：workitem_id,organization_id
def get_work_item_info(workitem_id,organization_id):
    work_item=client.get_work_item_info(workitem_id=workitem_id,organization_id=organization_id)
    return work_item.body.to_map()['workitem']

# 循环items_array，使用每个workitem的id，调用get_work_item_info方法，获取workitem的详细信息，并将结果保存到一个列表中
work_item_info_list=[]
for item in items_array:
    work_item_info_list.append(get_work_item_info(item['identifier'],organization_id))


In [ ]:
req = devops_20210625_models.ListWorkItemAllFieldsRequest(space_type="Project",
                                                  space_identifier=project_id,
                                                  workitem_type_identifier='37da3a07df4d08aef2e3b393')
res=client.list_work_item_all_fields_with_options(organization_id=organization_id,
                                                headers=headers,runtime=runtime,
                                                request=req)

# print(res.body)
# all_fields=json.loads(res.body)

fields_df=pd.DataFrame(res.body.to_map()['fields'])
fields_df.describe()

fields_dict = {}
for index, row in fields_df.iterrows():
    row_data = row.to_dict()  # Convert the row to a dictionary
    fields_dict[row_data['identifier']] = row_data['name']

fields_dict

In [ ]:
workitem_id_list=['a326b2de6dd6e58d2b4875a4cd', '603c94a0b6c81e0d96af8b90e1', 'c34bac818d755c391358b93fe4', '38a17bbf11429b89b87ecba814', '32c0421f3a60a9b18dde67867e', '0b32c5a5406f155442dc9bcda3', 'fb39c9b625aa44e206d85cd616', 'd55839901fd9e95c55c45ab74a', 'b5c616a364f6500b22ffde1a3c', '4be65b035d381eac8615045e24', '15a8b4125a788c60962a3291b0', '8d6f0b75c381ec83115cf51853', 'ac6c3d9782237f644ab470c820', '281d4bb7f9fefa80dfabf84949', '451063ef21a8ff19102426cf94', '05c87e8d263371e9ef35af26c4', '6de853d5bd7321d13f8eb2320b', '9ae6437f7814ec38372dcabd6c', '1d97d2eb2e39f1cc1d5972974b', 'eacbf3aaf7c04d42b71b236e35', 'ccfd3bd75c347e0c75087b2c9e', '4f3fe3cdc8317d50b6ffed6043', '7301aa23a6182c65950a48823a', '00174a722d692690c13b4f7223', 'e56e4ba1033141c3cb90609c4d', 'd38019d33d72b4baa260c1bcb5', 'bc192cbb85b94921001f8829b7']
work_item_info_list=[]
for id in workitem_id_list:
    workitem=get_work_item_info(workitem_id=id, organization_id=organization_id)
    comments=client.get_workitem_comment_list(organization_id=organization_id, workitem_id=id).body.to_map()['commentList']
    workitem['comments']=comments
    work_item_info_list.append(workitem)

work_item_info_list_df=pd.DataFrame(work_item_info_list)
work_item_info_list_df.head(1)

In [ ]:
# Set the option to display full column width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns
pd.set_option('display.width', None)  # Set width to None for automatic wrapping

In [ ]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
from markdownify import markdownify as md
import os
import socket

def get_local_ip():
    try:
        # Create a socket object
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        sock.connect(("8.8.8.8", 80))  # Connecting to a known external server (Google's DNS server)

        # Get the local IP address connected to the external server
        local_ip = sock.getsockname()[0]
        return local_ip
    except socket.error as e:
        return f"Error: {e}"

# Get and print your local IP
local_ip = get_local_ip()
print(f"Your local IP address is: {local_ip}")


ANTHROPIC_API_KEY=os.environ['ANTHROPIC_API_KEY']

proxies=f'http://{local_ip}:8001'
print(proxies)

anthropic = Anthropic(
    api_key=ANTHROPIC_API_KEY,
    proxies=proxies
)


def call_claude_get_suggestion(bug_info):
    prompt=f"""你是一个聪明的IT工程师，你会基于线上发现的bug，根据评论区的内容，分析每个BUG发生的原因，标记该BUG是否属于‘发布流程’问题，还是‘NullPointerExceptin’问题，还是疑难问题。同时你会给出该BUG的后续改进建议
你总是返回JSON格式的数据。
{HUMAN_PROMPT}标题：NPE:用户中心迁移过程漏掉了 tenant.type的过滤 
评论：https://sls.console.aliyun.com/lognext/project/k8s-log-c7d28cba17d0a416ca4f52459592b8d38/logsearch/prod-cosfo-oms-stdout-log?encode=base64&queryString=dHJhY2VJZDplYTFhNDEwMTRlMTY5NTEyMTMxOTg5NTEwNTdkMDAwMQ==&queryTimeType=99&startTime=1695120019&endTime=1695120619
客户请求到的租户详情 tenantid=1=鲜沐、
而鲜沐不应该是一个租户
用户中心迁移过程漏掉了 tenant.type的过滤  
详情和列表都应该有type的过滤
接口如下:
/tenant/query/detail
/tenant/query/list
{AI_PROMPT}{{"是否发布流程问题":"否","NullPointerExceptin":"是","是否疑难问题":"否","改进建议":"增强自测，提升单元测试的覆盖度"}}
{HUMAN_PROMPT}很好，很准确，请你继续返回JSON格式的回答。现在请继续分析:
标题：{bug_info['title']}
评论：{bug_info['comments']}
{AI_PROMPT}"""
    # print(prompt)
    completion = anthropic.completions.create(
        model="claude-2.1",
        max_tokens_to_sample=300,
        prompt=prompt,
        temperature=0.1,
    )
    # print(completion.completion)
    return completion.completion

In [ ]:
for item in work_item_info_list:
    comment_content=[]
    for cm in item['comments']:
        mdValue=md(json.loads(cm['content'])['htmlValue'], strip=['a'])
        comment_content.append(mdValue)

    print(item['subject'], "\n".join(comment_content))
    suggestion=call_claude_get_suggestion({"title":item['subject'],'comments':"\n".join(comment_content)})
    item['suggestion']=suggestion

In [ ]:
work_item_info_list_df=pd.DataFrame(work_item_info_list)
print(work_item_info_list_df.columns)

In [ ]:
work_item_info_list_df=work_item_info_list_df[['subject','serialNumber','suggestion','assignedTo']]
work_item_info_list_df

In [ ]:
str="""{"是否发布流程问题":"否","NullPointerExceptin":"否","是否疑难问题":"是","改进建议":"1. summerfarm-manage服务出现宕机,导致依赖其API的其他服务无法正常工作,需要检查manage服务的高可用性,例如多实例部署、健康检查等。

2. cosfo-erp直接硬编码依赖manage服务的API,一次依赖的不可用会导致自身的异常。可以考虑通过服务发现和熔断机制来提高容错性,如果manage服务不可用可以快速失败而不是异常。

3. 需要完善监控,可以设置manage服务的关键接口的监控指标,以便快速发现问题。"}"""

json.loads(str, strict=False)

In [ ]:
import json
work_item_info_list_df=pd.read_csv("./saas_q4_all_bugs.csv")

keys= ["是否发布流程问题", "NullPointerExceptin", "是否疑难问题", "改进建议"]

def apply_new_key_to_df(row, key):
    suggestion=row['suggestion']
    # print(row['subject'],suggestion)
    suggestion=json.loads(suggestion, strict=False)
    if key in suggestion:
        return suggestion[key]
    return ""

for key in keys:
    work_item_info_list_df[key] = work_item_info_list_df.apply(lambda row: apply_new_key_to_df(row, key), axis=1)

In [ ]:
work_item_info_list_df[['subject','serialNumber','是否发布流程问题','NullPointerExceptin','是否疑难问题','改进建议']].to_csv("./data/saas_q4_all_bugs_clean.csv", index=False)